# Website Scraping - National SAT Scores 

### Cleaning/Formatting Website Data

In [1]:
# Load packages and set options required for scraping website data
import pandas as pd
#import numpy as np
#import warnings
#warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import requests


Below we use requests.get to retrieve the text from the live webpage which contains our SAT data we wish to scrape. We use the status code function to verify our website was successfully read. Anything other than 200 would be cause for concern here. Then we can create our BeautifulSoup object and make use of its functions to read the website data and create a human readable table.

In [2]:
# Create variable for website
url = 'https://nces.ed.gov/programs/digest/d16/tables/dt16_226.40.asp'

# Read website
page = requests.get(url)

# Verify that website was successfully read
page.status_code

200

In [3]:
# Create BeautifulSoup Object
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
# Finding all instances of 'table' and use len(table) to find total count to ensure data has been successfully read
tables = soup.findAll('table')
len(tables)

8

If we carefully inspect the structure of our website and the variable tables, we can find that all the code for the table we want to scrape can be found under an element with a table class equal to "tabletop tableMain tableWidth". We can then use the find function to isolate just this table.

In [5]:
# Use soup.find to locate table with class "tabletop tableMain tableWidth"
sat_tables = soup.find('table', {'class' : 'tabletop tableMain tableWidth'})

To isolate the header for our final dataset, we inspected the structure of the webpage again and found that the relevant header information could be found in the 2nd `<tr>` element in the `<thead>` element. We employ `findAll` to isolate these header names.

In [6]:
'''Use findAll method to find instances of 'tr' in <thead> element
recursive=False means to not search children of this node for instances of 'tr'
findAll returns a ResultSet, it can be treated like a Python list
Use square bracket to find second instance only (this is where titles for each test type reside)
Display structure of test_types
'''
test_types = sat_tables.thead.findAll('tr', recursive=False,)[1]
test_types

<tr>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls003" scope="col">Writing score</th>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls003" scope="col">Writing score</th>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls003" scope="col">Writing score</th>
<th class="TblCls005" scope="col">Critical reading score</th>
<th class="TblCls005" scope="col">Mathe-<br/>matics score</th>
<th class="TblCls003" scope="col">Writing score</th>
<th class="TblCls003" colspan="2"

In [7]:
# Use list comprehension and findAll to isolate SAT test types and verify length
test_type_list = [th.getText().strip() for th in test_types.findAll('th')]
len(test_type_list)

18

In [8]:
# Display contents of test_type_list
test_type_list

['Critical reading score',
 'Mathe-matics score',
 'Critical reading score',
 'Mathe-matics score',
 'Critical reading score',
 'Mathe-matics score',
 'Writing score',
 'Critical reading score',
 'Mathe-matics score',
 'Writing score',
 'Critical reading score',
 'Mathe-matics score',
 'Writing score',
 'Critical reading score',
 'Mathe-matics score',
 'Writing score',
 '2005-06',
 '2014-15']

Unfortunately, we find that based on our scraping we have lost some very useful information from our webpage. With this in mind, we begin some of our transformations before creating the dataframe.

In [9]:
# Replace 'Writing score' 'Average SAT Writing' using list comprehension
replacement = 'Average SAT Writing'
original = 'Writing score'
test_type_list = [replacement if item==original else item for item in test_type_list]

In [10]:
# Replace 'Critical reading score' 'Average SAT Reading' using list comprehension
replacement = 'Average SAT Reading'
original = 'Critical reading score'
test_type_list = [replacement if item==original else item for item in test_type_list]

In [11]:
# Replace 'Mathe-matics score' 'Average SAT Math' using list comprehension
replacement = 'Average SAT Math'
original = 'Mathe-matics score'
test_type_list = [replacement if item==original else item for item in test_type_list]

In [12]:
# Display contents of test_type_list to verify header names have been updated
test_type_list

['Average SAT Reading',
 'Average SAT Math',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 '2005-06',
 '2014-15']

In [13]:
# Add missing column for 'State' as this is information which will be crucial for joining header and rows
test_type_list.insert(0,'State')
test_type_list

['State',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 'Average SAT Reading',
 'Average SAT Math',
 'Average SAT Writing',
 '2005-06',
 '2014-15']

In [14]:
# Update name of '2005-96' '2014-15'  to include 'Percent Tested '
test_type_list[18] = 'Percent Tested 2014-15'
test_type_list[17] = 'Percent Tested 2005-06'

The remaining columns which are missing the year, we will treat in similar fashion, calling them by their index and updating before we verify all column names are updated.

In [15]:
# For 1995-96 school year
# Rename 3rd list item 'Average SAT Math'  to 'Average SAT Math 1995-96'
test_type_list[2] = 'Average SAT Math 1995-96'

# Rename 2nd list item 'Average SAT Reading'  to 'Average SAT Reading 1995-96'
test_type_list[1] = 'Average SAT Reading 1995-96'

# For 2000-01 school year
# Rename 5th list item 'Average SAT Math'  to 'Average SAT Math 2000-01'
test_type_list[4] = 'Average SAT Math 2000-01'

# Rename 4th list item 'Average SAT Reading'  to 'Average SAT Reading 2000-01'
test_type_list[3] = 'Average SAT Reading 2000-01'

# For 2005-06 school year
# Rename 8th list item 'Average SAT Writing'  to 'Average SAT Writing 2005-06'
test_type_list[7] = 'Average SAT Writing 2005-06'

# Rename 7th list item 'Average SAT Math'  to 'Average SAT Math 2005-06'
test_type_list[6] = 'Average SAT Math 2005-06'

# Rename 6th list item 'Average SAT Reading'  to 'Average SAT Reading 2005-06'
test_type_list[5] = 'Average SAT Reading 2005-06'

# For 2010-11 school year
# Rename 11th list item 'Average SAT Writing'  to 'Average SAT Writing 2010-11'
test_type_list[10] = 'Average SAT Writing 2010-11'

# Rename 10th list item 'Average SAT Math'  to 'Average SAT Math 2010-11'
test_type_list[9] = 'Average SAT Math 2010-11'

# Rename 9th list item 'Average SAT Reading'  to 'Average SAT Reading 2010-11'
test_type_list[8] = 'Average SAT Reading 2010-11'

# For 2014-15 school year
# Rename 14th list item 'Average SAT Writing'  to 'Average SAT Writing 2014-15'
test_type_list[13] = 'Average SAT Writing 2014-15'

# Rename 13th list item 'Average SAT Math'  to 'Average SAT Math 2014-15'
test_type_list[12] = 'Average SAT Math 2014-15'

# Rename 12th list item 'Average SAT Reading'  to 'Average SAT Reading 2014-15'
test_type_list[11] = 'Average SAT Reading 2014-15'

# For 2015-16 school year
# Rename 17th list item 'Average SAT Writing'  to 'Average SAT Writing 2015-16'
test_type_list[16] = 'Average SAT Writing 2015-16'

# Rename 16th list item 'Average SAT Math'  to 'Average SAT Math 2015-16'
test_type_list[15] = 'Average SAT Math 2015-16'

# Rename 15th list item 'Average SAT Reading'  to 'Average SAT Reading 2015-16'
test_type_list[14] = 'Average SAT Reading 2015-16'
test_type_list

['State',
 'Average SAT Reading 1995-96',
 'Average SAT Math 1995-96',
 'Average SAT Reading 2000-01',
 'Average SAT Math 2000-01',
 'Average SAT Reading 2005-06',
 'Average SAT Math 2005-06',
 'Average SAT Writing 2005-06',
 'Average SAT Reading 2010-11',
 'Average SAT Math 2010-11',
 'Average SAT Writing 2010-11',
 'Average SAT Reading 2014-15',
 'Average SAT Math 2014-15',
 'Average SAT Writing 2014-15',
 'Average SAT Reading 2015-16',
 'Average SAT Math 2015-16',
 'Average SAT Writing 2015-16',
 'Percent Tested 2005-06',
 'Percent Tested 2014-15']

Now that we have spent a considerable amount of time cleaning up what will eventually become our header names, we will have to extract the rows for our dataset. We can find those located in the `<td>` elements within the `<tr>` elements in `<tbody>`. Below we employ list comprehension and the findAll and getText functions to strip the data in these elements to just the text we want in our table.

In [16]:
# Obtaining testing score data from <tbody> element
all_test_data = [td.getText().strip() for td in sat_tables.tbody.findAll('tr', recursive=False,)]

Unfortunately, the resulting line of code gives us a list where each element is a string representing entire row. Each piece of information we require is separated by a new line character. We will then need to split along these new line characters to create our list.

In [17]:
# Split on new line character for each list item in all_test_data
all_test_data
all_test_data_list = [item.splitlines() for item in all_test_data]

If we display this list, we will find some additional text which needs to be removed. Below we employ list comprehension to create an updated data list.

In [18]:
# Remove instances of '\xa0' from all_test_data_list
remove = '\xa0'
updated_test_data_list = [[ele for ele in sub if ele != remove] for sub in all_test_data_list]

There is regrettably, still data which needs to be cleaned before we can proceed wtih merging this data with the header names. We have a few empty rows which have been stored in the list as empty lists and we also need to remove the list created for the line 'District of'. We can verify we will have the correct number of total rows by checking the length of this cleaned data is 52 or each state plus the instances for the District of Colombia and the US average.

In [19]:
# Remove empty lists from updated_all_data_list
cleaned_test_data = [ele for ele in updated_test_data_list if ele != []]
cleaned_test_data = [ele for ele in cleaned_test_data if ele != ['District of']]

In [20]:
# Verify len of cleaned_test_data is 52 (all states, District of Columbia, and US average)
len(cleaned_test_data)

52

However, if we try to create the dataframe now, we will receive an error message because there will be at least one instance in which there are 20 columns in a row where we only have 19 headers. This is because when we used getText, the superscript for Percent Taking SAT in 2005-2006 for the District of Columbia was also included in our data. To address this issue, we will insert a column which will ultimately be dropped in the final dataset.

In [21]:
# Add column to be dropped due extra columns in cleaned_test_data 
test_type_list.insert(18, 'to be dropped')

Now we can create and view the dataset, but there are still a few steps before we will have our final dataframe.

In [22]:
# Create dataframe with scrapped test data assigned to variable cleaned_test_data with test_type_list as column names
cleaned_test = pd.DataFrame(cleaned_test_data, columns=test_type_list)

# Display first 5 rows
cleaned_test.head()

,State,Average SAT Reading 1995-96,Average SAT Math 1995-96,Average SAT Reading 2000-01,Average SAT Math 2000-01,Average SAT Reading 2005-06,Average SAT Math 2005-06,Average SAT Writing 2005-06,Average SAT Reading 2010-11,Average SAT Math 2010-11,Average SAT Writing 2010-11,Average SAT Reading 2014-15,Average SAT Math 2014-15,Average SAT Writing 2014-15,Average SAT Reading 2015-16,Average SAT Math 2015-16,Average SAT Writing 2015-16,Percent Tested 2005-06,to be dropped,Percent Tested 2014-15
0,United States,505,508,506,514,503,518,497,497,514,489,495,511,484,494,508,482,48,49,None
1,Alabama,565,558,559,554,565,561,565,546,541,536,545,538,533,557,551,543,9,6,None
2,Alaska,521,513,514,510,517,517,493,515,511,487,509,503,482,485,479,460,51,52,None
3,Arizona,525,521,523,525,521,528,507,517,523,499,523,527,502,528,532,505,32,34,None
4,Arkansas,566,550,562,550,574,568,567,568,570,554,568,569,551,570,569,553,5,4,None


As previously mentioned, we will need to fill in the missing data which we created by adding the 'to be dropped' column. 

In [23]:
# Fill in missing data in 'Percent Tested 2014-15' with the data from the 'to be dropped' column
cleaned_test['Percent Tested 2014-15'] = cleaned_test['Percent Tested 2014-15'].fillna(cleaned_test['to be dropped'])

# Display first 5 rows
cleaned_test.head()

,State,Average SAT Reading 1995-96,Average SAT Math 1995-96,Average SAT Reading 2000-01,Average SAT Math 2000-01,Average SAT Reading 2005-06,Average SAT Math 2005-06,Average SAT Writing 2005-06,Average SAT Reading 2010-11,Average SAT Math 2010-11,Average SAT Writing 2010-11,Average SAT Reading 2014-15,Average SAT Math 2014-15,Average SAT Writing 2014-15,Average SAT Reading 2015-16,Average SAT Math 2015-16,Average SAT Writing 2015-16,Percent Tested 2005-06,to be dropped,Percent Tested 2014-15
0,United States,505,508,506,514,503,518,497,497,514,489,495,511,484,494,508,482,48,49,49
1,Alabama,565,558,559,554,565,561,565,546,541,536,545,538,533,557,551,543,9,6,6
2,Alaska,521,513,514,510,517,517,493,515,511,487,509,503,482,485,479,460,51,52,52
3,Arizona,525,521,523,525,521,528,507,517,523,499,523,527,502,528,532,505,32,34,34
4,Arkansas,566,550,562,550,574,568,567,568,570,554,568,569,551,570,569,553,5,4,4


Then we can drop this column from out dataset as it is no longer needed.

In [24]:
# Remove column now that we have used it to fill the next column

final_test_df = cleaned_test.drop('to be dropped', axis=1)

One of the things which we have not addressed is that in the initial scrapped data, some of the state names have numbers and we need to update 'Columbia' to 'District of Columbia'.

In [25]:
# Remove numbers from state names in State column
final_test_df['State'].replace('\d+', '', regex=True, inplace=True)

# Add 'District of' to 'Columbia'

final_test_df['State'].replace('Columbia', 'District of Columbia', inplace=True)

### Display Final Dataset

In [26]:
# Display first 10 lines
final_test_df.head(10)

,State,Average SAT Reading 1995-96,Average SAT Math 1995-96,Average SAT Reading 2000-01,Average SAT Math 2000-01,Average SAT Reading 2005-06,Average SAT Math 2005-06,Average SAT Writing 2005-06,Average SAT Reading 2010-11,Average SAT Math 2010-11,Average SAT Writing 2010-11,Average SAT Reading 2014-15,Average SAT Math 2014-15,Average SAT Writing 2014-15,Average SAT Reading 2015-16,Average SAT Math 2015-16,Average SAT Writing 2015-16,Percent Tested 2005-06,Percent Tested 2014-15
0,United States,505,508,506,514,503,518,497,497,514,489,495,511,484,494,508,482,48,49
1,Alabama,565,558,559,554,565,561,565,546,541,536,545,538,533,557,551,543,9,6
2,Alaska,521,513,514,510,517,517,493,515,511,487,509,503,482,485,479,460,51,52
3,Arizona,525,521,523,525,521,528,507,517,523,499,523,527,502,528,532,505,32,34
4,Arkansas,566,550,562,550,574,568,567,568,570,554,568,569,551,570,569,553,5,4
5,California,495,511,498,517,501,518,501,499,515,499,495,506,491,491,500,485,49,60
6,Colorado,536,538,539,542,558,564,548,570,573,556,582,587,567,587,589,571,26,12
7,Connecticut,507,504,509,510,512,516,511,509,513,513,504,506,504,500,500,497,84,89
8,Delaware,508,495,501,499,495,500,484,489,490,476,462,461,445,458,453,440,73,100
9,District of Columbia,489,473,482,474,487,472,482,469,457,459,441,440,432,433,433,419,78,100


In [27]:
# Display last 10 lines
final_test_df.tail(10)

,State,Average SAT Reading 1995-96,Average SAT Math 1995-96,Average SAT Reading 2000-01,Average SAT Math 2000-01,Average SAT Reading 2005-06,Average SAT Math 2005-06,Average SAT Writing 2005-06,Average SAT Reading 2010-11,Average SAT Math 2010-11,Average SAT Writing 2010-11,Average SAT Reading 2014-15,Average SAT Math 2014-15,Average SAT Writing 2014-15,Average SAT Reading 2015-16,Average SAT Math 2015-16,Average SAT Writing 2015-16,Percent Tested 2005-06,Percent Tested 2014-15
42,South Dakota,574,566,577,582,590,604,578,584,591,562,592,597,564,586,581,558,4,3
43,Tennessee,563,552,562,553,573,569,572,575,568,567,581,574,568,586,582,571,15,7
44,Texas,495,500,493,499,491,506,487,479,502,465,470,486,454,466,478,449,52,64
45,Utah,583,575,575,570,560,557,550,563,559,545,579,575,554,579,579,558,7,5
46,Vermont,506,500,511,506,513,519,502,515,518,505,523,524,507,520,520,501,67,61
47,Virginia,507,496,510,501,512,513,500,512,509,495,518,516,499,520,517,498,73,72
48,Washington,519,519,527,527,527,532,511,523,529,508,502,510,484,501,506,481,54,66
49,West Virginia,526,506,527,512,519,510,515,514,501,497,509,497,495,525,511,502,20,15
50,Wisconsin,577,586,584,596,588,600,577,590,602,575,591,605,575,605,618,588,6,4
51,Wyoming,544,544,547,545,548,555,537,572,569,551,589,586,562,603,600,587,10,3


### Implications

Very minimal information was dropped during the course of web scraping and that which was removed was removed with the intention of making the dataset more readable and accurate, such as dropping numbers from strings for state names or the removal of text which served no functional analytical value, such as removing \xa0 from text retrieved in the table body. While one of the steps in completed involved the creation then deletion of a column from the dataset, the important information was preserved by filling in values which would have otherwise remained empty in the last column. Had this step not taken place, we would be missing very important information which establishes a relationship with the data used alongside another project and would not have a full and complete picture of the relationships between the various states and the proportion of the student population which took the SATs during the 2014-15 school year.

In [28]:
# Save cleaned df as csv
final_test_df.to_csv('cleaned_test.csv', index=False) 